In [68]:
import os
import copy 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.interpolation import shift
pd.set_option('display.max_columns', None)
%matplotlib inline

In [69]:
DATA_DIR = os.path.join('..', 'data')
os.listdir(DATA_DIR)

['history.csv', 'submission.csv', 'employees.csv']

In [70]:
hist = pd.read_csv(os.path.join(DATA_DIR, 'history.csv'))
empls = pd.read_csv(os.path.join(DATA_DIR, 'employees.csv'))
sbmsn = pd.read_csv(os.path.join(DATA_DIR, 'submission.csv'))

In [71]:
empls.head()

,EmployeeID,HiringDate,DismissalDate
0,01EDC1A7-5DA7-4939-854B-0002E0DF548D,9/1/18,NaN
1,91C351D9-38A1-40CB-9B69-000774211859,1/1/18,2/1/19
2,D699327D-8A78-4FBE-9CEA-001355717D3F,2/1/18,NaN
3,22102E27-5EEA-44F9-8632-0017B04B1236,10/1/14,5/1/18
4,08C0AFA9-ABC3-4FAF-8DE2-0018AE9A69F4,9/1/10,NaN


In [72]:
len(empls)

5373

In [73]:
empls.EmployeeID.nunique()

5373

In [74]:
empls[empls['DismissalDate'].isnull()]

,EmployeeID,HiringDate,DismissalDate
0,01EDC1A7-5DA7-4939-854B-0002E0DF548D,9/1/18,NaN
2,D699327D-8A78-4FBE-9CEA-001355717D3F,2/1/18,NaN
4,08C0AFA9-ABC3-4FAF-8DE2-0018AE9A69F4,9/1/10,NaN
6,FA5F9156-0C2E-4462-B045-002E6E259B49,3/1/18,NaN
8,F080C1DE-1803-461A-AF0B-004E19CBD634,6/1/18,NaN
...,...,...,...
5367,9634B9D2-6519-4345-AD76-FFA34A10A26D,3/1/15,NaN
5369,4D599D15-30AD-461B-B35A-FFB29D5F3A57,3/1/17,NaN
5370,F7A06FC0-8117-495D-BA4B-FFD4B9306DA2,9/1/14,NaN
5371,6204EE4F-26E3-437B-85C1-FFED4CC7B6C8,4/1/16,NaN


In [75]:
set(sbmsn.EmployeeID) - set(empls.EmployeeID)

set()

In [76]:
df = hist.merge(empls)

In [77]:
df

,EmployeeID,Date,DevCenterID,SBUID,PositionID,PositionLevel,IsTrainee,LanguageLevelID,CustomerID,ProjectID,IsInternalProject,Utilization,HourVacation,HourMobileReserve,HourLockedReserve,OnSite,CompetenceGroupID,FunctionalOfficeID,PaymentTypeId,BonusOneTime,APM,WageGross,MonthOnPosition,MonthOnSalary,HiringDate,DismissalDate
0,00116D71-E87D-4B64-A566-1F29B2A798A8,7/1/17,3,292,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.7619,40,0,0,0,15,1,9,0,39,0.887446,1,1,6/1/13,NaN
1,00116D71-E87D-4B64-A566-1F29B2A798A8,8/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,1.0000,0,0,0,0,15,1,9,200,28,0.887446,2,2,6/1/13,NaN
2,00116D71-E87D-4B64-A566-1F29B2A798A8,9/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.5000,72,0,0,0,15,1,9,0,43,0.887446,3,3,6/1/13,NaN
3,00116D71-E87D-4B64-A566-1F29B2A798A8,10/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.8636,16,0,0,0,15,1,9,0,49,0.887446,4,4,6/1/13,NaN
4,00116D71-E87D-4B64-A566-1F29B2A798A8,11/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.8000,8,0,0,0,15,1,9,0,43,0.887446,5,5,6/1/13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87761,FFF3B179-1D20-40FF-A330-A051BDF37301,10/1/18,1,295,2,1,1,18,9417F950-B661-48E8-84CF-8DDB1F8FDB50,56FF7210-52AF-4E4D-BEE9-3411E73BF063,0,1.0511,0,0,0,0,22,0,22,0,55,0.255562,1,1,5/1/18,NaN
87762,FFF3B179-1D20-40FF-A330-A051BDF37301,11/1/18,1,295,2,1,1,18,9417F950-B661-48E8-84CF-8DDB1F8FDB50,56FF7210-52AF-4E4D-BEE9-3411E73BF063,0,1.0326,8,0,0,0,22,0,22,0,55,0.255562,2,2,5/1/18,NaN
87763,FFF3B179-1D20-40FF-A330-A051BDF37301,12/1/18,1,295,2,1,0,18,9417F950-B661-48E8-84CF-8DDB1F8FDB50,56FF7210-52AF-4E4D-BEE9-3411E73BF063,0,1.0284,0,0,0,0,22,0,22,0,55,0.255562,3,3,5/1/18,NaN
87764,FFF3B179-1D20-40FF-A330-A051BDF37301,1/1/19,1,295,2,1,0,18,9417F950-B661-48E8-84CF-8DDB1F8FDB50,56FF7210-52AF-4E4D-BEE9-3411E73BF063,0,1.0250,0,0,0,0,22,0,22,0,51,0.306113,4,1,5/1/18,NaN


In [96]:
def get_prev_values_stats(df, column):
    employees = df.EmployeeID.unique()
    diff_values_emps = []
    lag_values_emps = []
    for empl in employees:
        column_values = np.array(df[df.EmployeeID == empl][column].values)
        diffs = np.insert(np.diff(column_values), 0, np.nan)
        diff_values_emps += list(diffs)
        lag_values_emps += list(shift(column_values, 1, cval=np.nan))
    return diff_values_emps, lag_values_emps

In [97]:
def get_num_feat_values(df, column):
    employees = df.EmployeeID.unique()
    maxs = []
    mins = []
    stds = []
    means = []
    last_minus_mins = []
    last_minus_maxs = []
    last_minus_means = []
    for empl in employees:
        column_values = np.array(df[df.EmployeeID == empl][column].values)
        for i in range(len(column_values)):
            if i == 0:
                maxs.append(np.nan) 
                mins.append(np.nan) 
                stds.append(np.nan) 
                means.append(np.nan) 
                last_minus_mins.append(np.nan) 
                last_minus_maxs.append(np.nan) 
                last_minus_means.append(np.nan)
            else:
                sub_list = column_values[:i]
                last_val = sub_list[len(sub_list) - 1]
                maxx = np.max(sub_list)
                minn = np.min(sub_list)
                stdd = np.std(sub_list)
                avgg = np.mean(sub_list)
                maxs.append(maxx) 
                mins.append(minn) 
                stds.append(stdd) 
                means.append(avgg) 
                last_minus_mins.append(last_val - maxx) 
                last_minus_maxs.append(last_val - minn) 
                last_minus_means.append(last_val - avgg)
    return maxs, mins, stds, means, last_minus_mins, last_minus_maxs, last_minus_means

In [98]:
def get_all_feat_values(df, column):
    employees = df.EmployeeID.unique()
    nuniques = []
    nunique_fracs = []
    time_since_last_change_vals = []
    for empl in employees:
        column_values = np.array(df[df.EmployeeID == empl][column].values)
        for i in range(len(column_values)):
            if i == 0:
                nuniques.append(np.nan)
                nunique_fracs.append(np.nan)
                time_since_last_change_vals.append(np.nan)
            else:
                sub_list = column_values[:i]
                n_unique = len(np.unique(sub_list))
                nuniques.append(n_unique)
                nunique_fracs.append(n_unique / len(sub_list))
                not_eq = list(np.argwhere(np.array(sub_list) != sub_list[-1]))
                if not_eq:
                    count_changes = np.argwhere(np.array(sub_list) != sub_list[-1])[0][0]
                else:
                    count_changes = 0
                time_since_last_change_vals.append(count_changes)
    return nuniques, nunique_fracs, time_since_last_change_vals

In [99]:
df['Wage_Gross_diff'], df['Wage_Gross_lag'] = get_prev_values_stats(df, 'WageGross')

In [100]:
df

,EmployeeID,Date,DevCenterID,SBUID,PositionID,PositionLevel,IsTrainee,LanguageLevelID,CustomerID,ProjectID,IsInternalProject,Utilization,HourVacation,HourMobileReserve,HourLockedReserve,OnSite,CompetenceGroupID,FunctionalOfficeID,PaymentTypeId,BonusOneTime,APM,WageGross,MonthOnPosition,MonthOnSalary,HiringDate,DismissalDate,Wage_Gross_diff,Wage_Gross_lag
0,00116D71-E87D-4B64-A566-1F29B2A798A8,7/1/17,3,292,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.7619,40,0,0,0,15,1,9,0,39,0.887446,1,1,6/1/13,NaN,NaN,NaN
1,00116D71-E87D-4B64-A566-1F29B2A798A8,8/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,1.0000,0,0,0,0,15,1,9,200,28,0.887446,2,2,6/1/13,NaN,0.000000,0.887446
2,00116D71-E87D-4B64-A566-1F29B2A798A8,9/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.5000,72,0,0,0,15,1,9,0,43,0.887446,3,3,6/1/13,NaN,0.000000,0.887446
3,00116D71-E87D-4B64-A566-1F29B2A798A8,10/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.8636,16,0,0,0,15,1,9,0,49,0.887446,4,4,6/1/13,NaN,0.000000,0.887446
4,00116D71-E87D-4B64-A566-1F29B2A798A8,11/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.8000,8,0,0,0,15,1,9,0,43,0.887446,5,5,6/1/13,NaN,0.000000,0.887446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87761,FFF3B179-1D20-40FF-A330-A051BDF37301,10/1/18,1,295,2,1,1,18,9417F950-B661-48E8-84CF-8DDB1F8FDB50,56FF7210-52AF-4E4D-BEE9-3411E73BF063,0,1.0511,0,0,0,0,22,0,22,0,55,0.255562,1,1,5/1/18,NaN,0.046507,0.209055
87762,FFF3B179-1D20-40FF-A330-A051BDF37301,11/1/18,1,295,2,1,1,18,9417F950-B661-48E8-84CF-8DDB1F8FDB50,56FF7210-52AF-4E4D-BEE9-3411E73BF063,0,1.0326,8,0,0,0,22,0,22,0,55,0.255562,2,2,5/1/18,NaN,0.000000,0.255562
87763,FFF3B179-1D20-40FF-A330-A051BDF37301,12/1/18,1,295,2,1,0,18,9417F950-B661-48E8-84CF-8DDB1F8FDB50,56FF7210-52AF-4E4D-BEE9-3411E73BF063,0,1.0284,0,0,0,0,22,0,22,0,55,0.255562,3,3,5/1/18,NaN,0.000000,0.255562
87764,FFF3B179-1D20-40FF-A330-A051BDF37301,1/1/19,1,295,2,1,0,18,9417F950-B661-48E8-84CF-8DDB1F8FDB50,56FF7210-52AF-4E4D-BEE9-3411E73BF063,0,1.0250,0,0,0,0,22,0,22,0,51,0.306113,4,1,5/1/18,NaN,0.050551,0.255562


In [101]:
len(df[df['Wage_Gross_diff'].notna()])

82393

In [148]:
wdf = pd.read_csv('../submission_ens_5.csv')

In [149]:
sum(wdf.target)

1581

In [150]:
wdf2 = pd.read_csv('../2022_submission_NAZARII_FEATURES.csv')

In [151]:
sum(wdf2.target == wdf.target)

3275